# Convert foreign payments to USD with historical FX rates

In [1]:
import os
import pandas as pd

from datetime import datetime
from datetime import timedelta

import quandl

In [2]:
if 'QUANDL_KEY' in os.environ:
    quandl.ApiConfig.api_key = os.environ['QUANDL_KEY']

In [3]:
eur_quandl_calls = {
    'ECB': 'ECB/EURUSD',
    'FRED': 'FRED/DEXUSEU',
    'BOE': 'BOE/XUDLSER'
    }

quandl_calls = {'EUR': eur_quandl_calls}

In [4]:
def get_fx_rate_smoothed(date_string, fx_call_code):
    '''
    Gets fx rate for date according to fx call code, if missing, takes the average
    of fx rates before and after
    '''
    
    date_stamp = datetime.strptime(date_string, '%Y-%m-%d')
    
    try:
        fx_rate = quandl.get(fx_call_code, 
                             start_date=date_stamp, end_date=date_stamp + timedelta(days=1))
    except quandl.errors.quandl_error.NotFoundError as err:
        print(err)
        return
    
    if len(fx_rate) != 0:
        return fx_rate['Value'][0]
    else:
        return smooth_fx_rate(date_stamp, fx_call_code)
    
    
def smooth_fx_rate(date_stamp, quandl_call_code):
    ''''''
    fx_rate_before = quandl.get(quandl_call_code, 
                                    start_date = date_stamp - timedelta(days=7),
                                    end_date = date_stamp - timedelta(days=1))
    fx_rate_after = quandl.get(quandl_call_code, start_date=date_stamp + timedelta(days=1), 
                                         end_date=date_stamp + timedelta(days=7))
        
    return (fx_rate_before['Value'][-1] + fx_rate_after['Value'][0])/2


salary = pd.read_csv('example.csv')
date_string=salary['Date'][0]
fx_call_code = quandl_calls['EUR']['ECB']
get_fx_rate_smoothed(date_string, fx_call_code)

1.0632

In [5]:
fx_call_code =quandl_calls['EUR']['ECB']
salary['fx_rate'] = salary['Date'].apply(lambda date_string: 
                                          get_fx_rate_smoothed(date_string, fx_call_code))
salary['amount_usd'] = salary['Amount']*salary['fx_rate']
salary

,Date,Amount,fx_rate,amount_usd
0,2017-01-20,1000,1.0632,1063.2
1,2017-02-20,1000,1.0616,1061.6
2,2017-03-20,1500,1.0752,1612.8


In [6]:
def convert_payments(payment_csv_path, amount_col='Amount', date_col = 'Date',
                       currency='EUR', source='ECB'):
    payment_df = pd.read_csv(payment_csv_path)
    
    return convert_payment_df(payment_df, amount_col, date_col, currency, source)


def convert_payment_df(payment_df, amount_col='Amount', date_col = 'Date',
                       currency='EUR', source='ECB'):
    fx_call_code = quandl_calls[currency][source]
    
    payment_df['fx_rate'] = salary[date_col].apply(lambda date_string: 
                                          get_fx_rate_smoothed(date_string, fx_call_code))
    
    payment_df['amount_usd'] = payment_df[amount_col]*payment_df['fx_rate']

    return payment_df

convert_payments('example.csv')

,Date,Amount,fx_rate,amount_usd
0,2017-01-20,1000,1.0632,1063.2
1,2017-02-20,1000,1.0616,1061.6
2,2017-03-20,1500,1.0752,1612.8
